# Use public data to replicate S.Chevrot (2000) *Earth and Planetary Science Letters* results Figure 3

## Imports, global setup

In [ ]:
import os

import numpy as np
import pandas as pd

import rf
import rf.imaging

import matplotlib.pyplot as plt
import seaborn as sns

from obspy import UTCDateTime

In [ ]:
import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

## Load the data file of processed RF traces for analysis

In [ ]:
rf_type = 'LQT_fd'
# rf_type = 'LQT_td'
model = 'iasp91'
# model = 'ak135'
data = rf_util.read_h5_rf(r"..\DATA\7B_rfs_19930503T030058-19950810T012516_{}_{}_rev2_qual.h5".format(rf_type, model))
data

In [ ]:
# primary_station = 'SD02'
primary_station = 'SA01'

In [ ]:
# stations = set([tr.stats.station for tr in data])
# station_idx = {st: data.select(station=st) for st in stations}
# len(station_idx)

In [ ]:
# stations_not_empty = set([st for st in station_idx if len(station_idx[st]) > 0])
# len(stations_not_empty)

In [ ]:
# print(sorted(stations_not_empty))

## Process data and present RF stacking

### Filter data down to only RF traces (filter out raw traces)

In [ ]:
data_rf = data.select(station=primary_station)
len(data_rf)

In [ ]:
# Preview first 100 RF plots
_ = rf_plot_utils.plot_rf_stack(data_rf.sort(['back_azimuth'])[0:100], time_window=(-5,30))

### Check the main data channel code and set channel accordingly

In [ ]:
set([tr.stats.channel for tr in data_rf])

In [ ]:
channel = 'BHQ'

In [ ]:
importlib.reload(rf_util)

### Filter RFs to those with good enough SNR

In [ ]:
rf_util.label_rf_quality_simple_amplitude(rf_type, data_rf, snr_cutoff=1.7)
data_good = rf.RFStream([tr for tr in data_rf if tr.stats.predicted_quality == 'a']).sort(['back_azimuth'])
len(data_good)

### Plot the good RFs

In [ ]:
time_window=(-5.0, 30.0)

In [ ]:
save_file = 'RF_stack_{}.{}_{}_{}_validation.png'.format(primary_station, channel, rf_type, model)
fig = rf_plot_utils.plot_rf_stack(data_good, save_file=save_file, dpi=300, time_window=time_window)
plt.show()

In [ ]:
data_good_events = [tr.stats.event_id for tr in data_good]

## H-k stacking

In [ ]:
db = rf_util.rf_to_dict(data_good)
data_sta = db[primary_station]

In [ ]:
weighting = (0.5, 0.5, 0.0)
# weighting = (0.33, 0.33, 0.33)

V_p = 6.4
k_grid, h_grid, hk_stack = rf_stacking.compute_hk_stack(data_sta, channel, V_p=V_p, root_order=2)

# Sum the phases
hk_stack_sum = rf_stacking.compute_weighted_stack(hk_stack, weighting)

# Raise the final sum over phases to power >1 to increase contrast
hk_stack_sum = rf_util.signed_nth_power(hk_stack_sum, 2)
hk_stack_sum = hk_stack_sum/np.max(hk_stack_sum[:])

# Numerically find location of maximum
h_max, k_max = rf_stacking.find_global_hk_maximum(k_grid, h_grid, hk_stack_sum)
print("Numerical solution (H, k) = ({:.3f}, {:.3f})".format(h_max, k_max))

sta = data_sta[channel][0].stats.station
num = len(data_sta[channel])
save_file = 'Hk_stack_{}.{}_{}_{}_validation.png'.format(sta, channel, rf_type, model)
_ = rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack_sum, title='Station ' + sta + '.{}'.format(channel), num=num, save_file=save_file)

## Further analysis to disambiguate which is the "correct" selection of *H-k* maximum

### Generate simple picks on $t_1$, $t_2$ and use analytic solution for $(H, \kappa)$ to compute scattergram based on picks

In [ ]:
# Get t1, t2 from picking maxima within narrow time bands based on RF plot above
t1 = []
t2 = []
incl = []
slowness_secperkm = []
km_per_deg = 111.1949
snr_ad = np.array([tr.stats.snr for tr in data_good])
for tr in data_good:
    t_offset = tr.stats.onset - tr.stats.starttime
    t_rel = tr.times() - t_offset
    t1_mask = ((t_rel >= 3) & (t_rel <= 5))
    t2_mask = ((t_rel >= 14) & (t_rel <= 16))
    t1_max = np.max(tr.data[t1_mask])
    t2_max = np.max(tr.data[t2_mask])
    t1_index = np.where(tr.data[t1_mask] == t1_max)
    t2_index = np.where(tr.data[t2_mask] == t2_max)
    t1_val = t_rel[t1_mask][t1_index]
    t2_val = t_rel[t2_mask][t2_index]
    t1.append(t1_val.mean())
    t2.append(t2_val.mean())
    incl.append(tr.stats.inclination*np.pi/180.0)
    slowness_secperkm.append(tr.stats.slowness/km_per_deg)
t1 = np.array(t1)
t2 = np.array(t2)
incl = np.array(incl)
slowness_secperkm = np.array(slowness_secperkm)
ray_param = np.sin(incl)*slowness_secperkm

In [ ]:
alpha = t2 - t1
beta = t1 + t2
print(V_p)

In [ ]:
p = ray_param
H = alpha/(2*np.sqrt(1/V_p**2 - p*p))
k = V_p*np.sqrt((beta/(2*H))**2 + p*p)

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

plt.figure(figsize=(13,12))
ax = plt.gca()
sns.scatterplot(k, H, hue=snr_ad, size=snr_ad, sizes=(50, 250), s=1000, alpha=0.9, ax=ax)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel(r'$\kappa = \frac{V_p}{V_s}$ (ratio)', fontsize=14)
plt.ylabel('H = Moho depth (km)', fontsize=14)
plt.grid(color="#80808080", linestyle=":")
plt.xlim(1.3, 2.1)
plt.ylim(20, 70)
plt.legend(title='SNR')
plt.title('Per-event $H-\kappa$ scattergram based on $t_1$, $t_2$ picks', fontsize=20)
# ellipse = Ellipse((0, 0), width=0.4, height=5, facecolor=None, edgecolor='#202020', linestyle='--', fill=False)
# transf = transforms.Affine2D().rotate_deg(0).translate(1.85, 39)
# ellipse.set_transform(transf + ax.transData)
# ax.add_patch(ellipse)
plt.savefig('Hk_scattergram_{}.{}_{}_{}_validation.png'.format(primary_station, channel, rf_type, model), dpi=300)
plt.show()

## Extend validation to cover other stations of Skippy deployment

In [ ]:
db_7b = rf_util.rf_to_dict(data)
test_stations = ['SA01', 'SA02', 'SA03', 'SA05', 'SA08', 'SB02', 'SB04', 'SB08', 'SC03', 'SC05','SC06',
                 'SC07', 'SC08', 'SC09', 'SD02', 'SD05', 'SE03', 'SE06', 'SF03', 'SF06', 'SF09', 'YB04']

In [ ]:
output_folder = 'schevrot_validation_rev2'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
for test_station in test_stations:
    try:
        db_station = db_7b[test_station]
        channel = rf_util.choose_rf_source_channel(rf_type, db_station)

        db_channel = db_station[channel]
        test_rf = rf.RFStream(db_channel)

        rf_util.label_rf_quality_simple_amplitude(rf_type, test_rf, snr_cutoff=1.7)
        data_good = rf.RFStream([tr for tr in test_rf if tr.stats.predicted_quality == 'a']).sort(['back_azimuth'])
        print("Num traces = {}".format(len(data_good)))

        save_file = 'RF_stack_{}.{}_{}_{}_validation.png'.format(test_station, channel, rf_type, model)
        save_file = os.path.join(output_folder, save_file)
        fig = rf_plot_utils.plot_rf_stack(data_good, save_file=save_file, dpi=300, time_window=time_window)

        db_good = rf_util.rf_to_dict(data_good)
        data_sta = db_good[test_station]

        weighting = (0.5, 0.5, 0.0)

        V_p = 6.4
        k_grid, h_grid, hk_stack = rf_stacking.compute_hk_stack(data_sta, channel, V_p=V_p, root_order=2)

        # Sum the phases
        hk_stack_sum = rf_stacking.compute_weighted_stack(hk_stack, weighting)

        # Raise the final sum over phases to power >1 to increase contrast
        hk_stack_sum = rf_util.signed_nth_power(hk_stack_sum, 2)
        hk_stack_sum = hk_stack_sum/np.max(hk_stack_sum[:])

        # Numerically find location of maximum
        h_max, k_max = rf_stacking.find_global_hk_maximum(k_grid, h_grid, hk_stack_sum)
        print("{}: Numerical solution (H, k) = ({:.3f}, {:.3f})".format(test_station, h_max, k_max))

        sta = test_station
        num = len(data_sta[channel])
        save_file = 'Hk_stack_{}.{}_{}_{}_validation.png'.format(sta, channel, rf_type, model)
        save_file = os.path.join(output_folder, save_file)
        _ = rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack_sum, title='Station ' + sta + '.{}'.format(channel), num=num, save_file=save_file)
        plt.show()
    except Exception as e:
        print("Failed on station {} with error:\n{}".format(test_station, str(e)))
    # end try
# end for

--------------

## APPENDIX: Derivation of solution for $(H, \kappa)$ from $(t_1, t_2)$

There are various similar and equivalent formulations for the theoretical arrival times $t_1, t_2$ of the Ps and PpPs phases respectively. The formulation chosen here is that of Youlin Chen *et al.*, "Crustal structure beneath China from receiver function analysis", *Journal of Geophysical Research*, Vol.115, B03307 (2010), doi:10.1029/2009JB006386.

Starting with:

$t_1 = H \left[\sqrt{\left(\frac{\kappa}{V_p}\right)^2 - p^2} - \sqrt{\left(\frac{1}{V_p}\right)^2 - p^2} \right] \qquad \text{Ps}$

and

$t_2 = H \left[\sqrt{\left(\frac{\kappa}{V_p}\right)^2 - p^2} + \sqrt{\left(\frac{1}{V_p}\right)^2 - p^2} \right] \qquad \text{PpPs}$

where $H$ = depth to Moho, $\kappa = \frac{V_p}{V_s}$, $V_p$ = P-wave velocity, $p$ = ray parameter (per ray/event).

The let $\alpha = t_2 - t_1$, $\beta = t_1 + t_2$. Algebraic solution yields

$\alpha = 2H\sqrt{\left(\frac{1}{V_p}\right)^2 - p^2}$

$\implies H = \frac{\alpha}{2\sqrt{\left(\frac{1}{V_p}\right)^2 - p^2}}$

in which one can see that $\alpha$ is independent of $\kappa$ and depends only on $H$.

For $\beta$,

$\beta = 2H\sqrt{\left(\frac{\kappa}{V_p}\right)^2 - p^2}$

$\implies \kappa = \frac{V_p}{V_s} = V_p \sqrt{\left(\frac{\beta}{2H}\right)^2 + p^2}$